# Intrinsic Valuation Functions

In [44]:
def country_def_spread(country):
    if country=='USA':
        return 0.0
    
def govt_bond_rate(country, currency):
    if country=='USA':
        if currency=='USD':
            return .0357

# Long term cost to borrow?
def pre_tax_rate(country):
    if country=='test':
        return .1
        
def tax_rate(country):
    if country=='test':
        return .5
    
def cod(country):
    return pre_tax_rate(country)*(1 - tax_rate(country))
    
def wacc(mkt_equity, coe, mkt_debt, codebt):
    return coe*mkt_equity/(mkt_equity + mkt_debt) + codebt*mkt_debt/(mkt_equity + mkt_debt)
    
    
def dcf(rate,flow,term_val):
    if len(rate)!=len(flow):
        raise Exception("Cash flows and rates must be same length")
    
    # Initialize value, add cash flows, add terminal value
    value=0
    for i,cf in enumerate(flow):
        value += flow[i]/(1 + rate[i])**(i+1)
    value += term_val/(1 + rate[len(flow)-1])**(len(flow))
    
    return value

# Country and Currency Parameters

In [31]:
country = 'USA'
currency = 'USD'

# Company Data

In [54]:
# https://pages.stern.nyu.edu/~adamodar/pdfiles/eqnotes/valpacket1spr23.pdf
# Slides 8-10

# Equity test case - cost of equity, CF to equity, equity terminal value
test_coe = .13625
test_coeq = [test_coe, test_coe, test_coe, test_coe, test_coe]
test_eq_flow = [50,60,68,76.2,83.49]
test_eq_tv = 1603.0
test_eq_val = dcf(test_coeq,test_eq_flow,test_eq_tv)

# Firm case - cost of capital, CF to firm, firm terminal value
test_mkt_debt = 800
test_cod = cod('test')
test_coc = wacc(test_eq_val, test_coe, test_mkt_debt, test_cod)
test_cocap = [test_coc, test_coc, test_coc, test_coc, test_coc]
test_firm_flow = [90,100,108,116.2,123.49]
test_firm_tv = 2363.008
test_firm_val = dcf(test_cocap,test_firm_flow,test_firm_tv)

# Value Calculations

In [55]:
print(test_eq_val)
print(test_firm_val)

1073.0065063585455
1873.4651511854781
